<a href="https://colab.research.google.com/github/OlaOye23/Mauctree/blob/main/Auctree_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase_admin
!pip install azure-storage-blob
AZR_CONN_STR = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#**Attempt 1: Scraping the list view **

**problems**:- no images, limited fields of info, only 5 propeties appeared

status:- largely resolved in attempt 2

In [ ]:
'''
#Python program to scrape website 
#and save data from website 
import requests 
from bs4 import BeautifulSoup
from IPython.core.display import HTML

styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

#GET HTML DOM contents
URL = "https://auctree.com/en/"#URL for getting BDC rates from abokiFX.com
r = requests.get(URL) #response to the HTTP request on the abokiFX site
soup = BeautifulSoup(r.content, 'html5lib') #'soupify' for traversal using bs4


body = soup.find('div', attrs = {'class': 'css-t23ds5'}) 

print(body)

names = []
names_ = body.findAll('span', attrs = {'id': 'SearchTableName'}) 
for name in names_:
  names.append(name.text)

prop_types = []
prop_types_ = body.findAll('span', attrs = {'id': 'SearchTablePropertyTypes'}) 
for prop_type in prop_types_:
  prop_types.append(prop_type.text)

regions = []
regions_ = body.findAll('span', attrs = {'id': 'SearchTableRegion'})
for region in regions_:
  regions.append(region.text)

cities = []
cities_ = body.findAll('span', attrs = {'id': 'SearchTableCities'})
for city in cities_:
  cities.append(city.text)

sale_types = []
sale_types_ = body.findAll('span', attrs = {'id': 'SearchTableSaleType'}) 
for sale_type in sale_types_:
  sale_types.append(sale_type.text)

auc_prices = []
auc_prices_ = body.findAll('span', attrs = {'id': 'SearchTableAuctionPrice'}) 
for auc_price in auc_prices_:
  auc_prices.append(auc_price.text)

time_lefts = []
time_lefts_ = body.findAll('span', attrs = {'id': 'SearchTableUntil'}) 
for time_left in time_lefts_:
  time_lefts.append(time_left.text)

end_dates = []
end_dates_ = body.findAll('span', attrs = {'id': 'SearchTableEndDate'}) 
for end_date in end_dates_:
  end_dates.append(end_date.text)

numbers = []
for name in names:
  number = name[:name.find('.')]
  numbers.append(number)

numbers = []
for i in range(len(names)):
  number = (names[i])[:names[i].find('.')]
  names[i] = (names[i])[names[i].find('Cl')+3:]# ignore number + cl prefix
  numbers.append(number)

images = []
for number in numbers:
  #GET HTML DOM contents
  URL = "https://auctree.com/en/auction/"+number #URL
  r = requests.get(URL) 
  soup = BeautifulSoup(r.content, 'html5lib')
  image = soup.find('img', attrs = {'class': 'MuiCardMedia-root MuiCardMedia-media MuiCardMedia-img css-o69gx8-MuiCardMedia-root'})['src']
  print(image)
  images.append(image)


import pandas as pd
prop_df = pd.DataFrame([numbers, names, images,  prop_types, regions, cities, sale_types, auc_prices, time_lefts, end_dates]).T
prop_df.columns = ['number', 'name', 'image', 'prop_type', 'region', 'city', 'sale_type', 'auc_price', 'time_left', 'end_date']
prop_dict = prop_df.T.to_dict()
print(prop_dict)


print(prop_df[:])

csv_name = '/content/auctree_data.csv'
prop_df.to_csv(csv_name, index= False)
json_name = '/content/auctree_data.json'
prop_df.T.to_json(json_name)
print('completed')


'''

'\n#Python program to scrape website \n#and save quotes from website \nimport requests \nfrom bs4 import BeautifulSoup\nfrom IPython.core.display import HTML\n\nstyles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text\nHTML(styles)\n\n#GET HTML DOM contents\nURL = "https://auctree.com/en/"#URL for getting BDC rates from abokiFX.com\nr = requests.get(URL) #response to the HTTP request on the abokiFX site\nsoup = BeautifulSoup(r.content, \'html5lib\') #\'soupify\' for traversal using bs4\n\n\nbody = soup.find(\'div\', attrs = {\'class\': \'css-t23ds5\'}) \n\nprint(body)\n\nnames = []\nnames_ = body.findAll(\'span\', attrs = {\'id\': \'SearchTableName\'}) \nfor name in names_:\n  names.append(name.text)\n\nprop_types = []\nprop_types_ = body.findAll(\'span\', attrs = {\'id\': \'SearchTablePropertyTypes\'}) \nfor prop_type in prop_types_:\n  prop_types.append(prop_type.text)\n\nregions = []\nregions_ = body.findAll(\'span\', a

**Attempt 2: Scraping individual property pages**

In [ ]:
#Python program to scrape website 
#and save data from website
import requests 
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import mechanize
import http

styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)


properties = []
#loop throught the pages and get relevant field values
for i in range(10,1500):
  #open the web page 
  URL = "https://auctree.com/en/auction/"+str(i) #URL
  r = requests.get(URL)
  soup = BeautifulSoup(r.content, 'html5lib')
  data = {}

  #retireve the field values
  try:
    #retrieve property number and name(address)
    name = soup.find('h4', attrs = {'class': 'MuiTypography-root MuiTypography-h4 css-xbbocf-MuiTypography-root'}).getText()
    data['number'] = name[:name.find(".")].strip()
    data['name'] = name[name.find(". ")+1:].strip()
  except Exception as e:
    data['number'] = ""
    data['name'] = ""
  try:
    #retrieve and clean region, city, and prop_type....*****NOTE: Region and city have been accidentally swapped ******FIX!
    location = soup.find('p', attrs = {'class': 'MuiTypography-root MuiTypography-body2 css-19mp1k9-MuiTypography-root'}).getText()
    data['region'] = location[:location.find(',')].strip()
    data['city'] = location[location.find(',')+1:location.find('·')].strip()
    data['prop_type'] = location[location.find('·')+1:].strip()
  except Exception as e:
    data['region'] = ""
    data['city'] = ""
    data['prop_type'] = ""
  try:
    #retrive time until auction end
    data['time_left'] = soup.find('p', attrs = {'class': 'MuiTypography-root MuiTypography-body2 css-1hv23l3-MuiTypography-root'}).getText()
  except Exception as e:
    data['time_left'] = ""
  try:
    #retrieve action close date
    data['close_date'] = soup.find('p', attrs = {'class': 'MuiTypography-root MuiTypography-body2 css-1hv23l3-MuiTypography-root'}).getText()
  except Exception as e:
    data['close_date'] = ""
  try:
    #retrive and clean start bid
    start_bid = soup.find('h6', attrs = {'class': 'MuiTypography-root MuiTypography-h6 css-t7sa6x-MuiTypography-root'}).getText()
    if start_bid[0] == '€':
      data['start_bid'] = float(start_bid[start_bid.find('€')+1:])
    elif start_bid[-1] == '€':
      data['start_bid'] = float(start_bid[:start_bid.find('\\xa0')-1])
  except Exception as e:
    data['start_bid'] = ""
  try:
    #retirve and clean deposit
    bod = soup.find('div', attrs = {'class': 'css-1mq1uye'})
    deposit = bod.find('p', attrs = {'class': 'MuiTypography-root MuiTypography-body2 css-19mp1k9-MuiTypography-root'}).getText()
    data['deposit'] = float((deposit[:deposit.find('%')]).replace(',','.'))/100
  except Exception as e:
    data['deposit'] = ""
  try:
    #retrieve asale type
    bod = soup.find('div', attrs = {'class': 'css-1ehc112'})
    data['sale_type'] = bod.find('span', attrs = {'class': 'MuiTypography-root MuiTypography-body3 MuiTypography-noWrap css-1r5mk5o-MuiTypography-root'}).getText()
  except Exception as e:
    data['sale_type'] = ""
  try:
    #retieve image source address
    data['image'] = soup.find('img', attrs = {'class': 'MuiCardMedia-root MuiCardMedia-media MuiCardMedia-img css-o69gx8-MuiCardMedia-root'})['src']
  except Exception as e:
    data['image'] = ""
  try:
    #retirve and clean buy it now price
    buy_now = soup.find('h6', attrs = {'class': 'MuiTypography-root MuiTypography-h6 css-859x9-MuiTypography-root'}).getText()
    if buy_now[0] == '€':
      data['buy_now'] = float(buy_now[buy_now.find('€')+1:])
    elif buy_now[-1] == '€':
      data['buy_now'] = float(buy_now[:buy_now.find('\\xa0')-1])
  except Exception as e:
    data['buy_now'] = ""
  properties.append(data)
  print(i)






**Upload to cloud storage for analytics in Synapse**

In [ ]:
from pandas.io.json import json_normalize
flat_data = json_normalize(properties)
#load json to csv
csv_filename = 'auctree_big.csv'
flat_data.to_csv(csv_filename)

#upload the data to Azure Storage
import os
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

try:
    print("Starting upload process...")
    # Retrieve the connection string for use with the application. The storage
    connect_str = AZR_CONN_STR
    # Create the BlobServiceClient object which will be used to create a container client
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)

    # Create a unique name for the container
    container_name = "mauctree" #+ str(uuid.uuid4())
    try:
      container_client = blob_service_client.create_container(container_name)
    except Exception as e:
      print('Storage container creation exception:')
      print(e)
    
    #upload the files
    
    try:
      # Create a file in the local data directory to upload and download
      local_path = '/content'
      local_file_name = csv_filename
      upload_file_path = os.path.join(local_path, local_file_name)

      # Create a blob client using the local file name as the name for the blob
      blob_client = blob_service_client.get_blob_client(container=container_name, blob=upload_file_path)

      print("\nUploading to Azure Storage as blob:\n\t" + local_file_name)

      # Upload the created file
      with open(upload_file_path, "rb") as data:
          blob_client.upload_blob(data)
          
    except Exception as e:
      print('Upload to container exception:')
      print(e)


    # List the blobs in the container
    print("\nListing blobs...")
    blob_list = container_client.list_blobs()
    for blob in blob_list:
        print("\t" + blob.name)
        
except Exception as e:
    print('Exception: ')
    print(e)

In [ ]:
# Use a service 
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import datetime 
import time
cred = credentials.Certificate('/content/shopmob-secret.json')
try:
  firebase_admin.initialize_app(cred)
except:
  print('error occurred in initialising firebase. Perhaps already initialised')

db = firestore.client()
now = datetime.datetime.now()
nowCalc = time.mktime(now.timetuple())
#post to db
for propertie in properties:
  response = db.collection(u'mauctree_properties').document().set(propertie)


In [ ]:
response